## 2. Fasea: Algoritmoak diseinatzen

#### [Ikasle]

Community Detection proiektuaren 1. fasea entregatu duzue, eta feedback-a jaso ere. Klasean hainbat algoritmo ikusi ditugu, batzuk soluzio bakarrean oinarritutakoak, beste batzuk aldiz, populazio bat erabiltzen dutenak. Horiez gain, hibridatzeko teknikak ere ikusi ditugu. Bigarrengo fase honetan, hiru algoritmo diseinatu beharko dituzue. Lehenengoa, algoritmo eraikitzaile bat izango da. Bigarrena, soluzio bakarrean oinarritutako heuristiko bat izan beharko du, eta azkenik, hirugarrena algoritmo poblazional bat izango da. Hiru algoritmoak estokastikoak izan beharko dute, eta horietatik, bik, oinarri probabilistikoa izan beharko dute. Adibidez, Simulated Annealing, Estimation of Distribution Algorithms (EDAk) edota Ant Colony Optimization (ACO) implementatu ditzazkezue. Proiektu honen kasuan, algoritmoen helburua, komunitate kopuru jakin bat emanik, modularitatea maximizatzen duen komunitate banaketa (soluzioa) bilatzen saiatzea da.

Errepasatu gaitegian zehar ikusi ditugun algoritmo guztiak, eta horiek kontuak izanik, libre zarete nahi dituzuen diseinuak sortzeko, baita ere hibridoak! Adi! Egiten duzuen aukeraketa argudiatu egin beharko duzue.

#### Entregablea

Bigarrengo fasea ebaluatu ahal izateko, notebook honetan bertan algoritmoen diseinua eta implementazioa proposatu beharko duzue. Gogoratu algoritmo bat azaltzeko modurik errezena fluxu diagrama eta sasikode bat egitea direla. Adi! Atal bakoitzean hartutako erabakiak eta garatutako metodoak egoki argudiatu beharko dituzue. Azalpenak ere nahi ditut. Diagramak ez dira eurak bakarrik azaltzen, beraz testutik erreferentziatu egin beharko dituzue. Saiatu idazkera zientifiko-tekniko batekin idazten (pentsatu publikatuko duzuen lan bat dela). Ez argudiatzeak edo lana garaiz ez entregatzeak penalizazioa jasoko dute ebaluagarria den proiektuaren zati honetan. eGelan zehazten dira notebook-a igotzeko <b>egun eta orduak</b>.

Momentuz, ez daukazue algoritmoen exekuzio eta konparaketak egin behar. Hirugarren fasean, esperimentazioaren inguruko baldintzak emango dizkizuet, eta, horrez gain, txostenaren idazketa burutu beharko duzue.

In [479]:
import pandas as pd
from itertools import product
# https://link.springer.com/chapter/10.1007/978-981-16-7502-7_29

import networkx as nx
import numpy as np
import pandas as pd
import sqlite3
import networkx.algorithms.community as nx_comm
import itertools
from operator import itemgetter
from time import time
import random
from numpy import exp
from numpy.random import randn
from numpy.random import rand
import math
def sortu_grafoa():

    # Datuak irakurri
    # BETE HEMEN 8 lerro
    connect = sqlite3.connect(r"database.sqlite")
    query = """
    SELECT pa.paper_id, pa.author_id, a.name
    FROM paper_authors AS pa JOIN papers AS p ON pa.paper_id = p.id
    JOIN authors as a ON pa.author_id = a.id
    WHERE p.Year BETWEEN '2014' AND '2015'
    """
    df = pd.read_sql(query, connect)

    
    # Sortu grafoa
    # BETE HEMEN 7-10 lerro
    G = nx.Graph()

    for p, a in df.groupby('paper_id')['name']: 
        for u, v in itertools.combinations(a, 2):
            if G.has_edge(u, v):
                G[u][v]['weight'] +=1
            else:
                G.add_edge(u, v, weight=1)
    return G


In [480]:
G = sortu_grafoa()
authors = np.array(G.nodes())

In [481]:
def modularitatea(G, partizioa, weight="weight"):
    degrees = dict(G.degree(weight=weight))
    lag = sum(degrees.values())
    m = lag / 2
    norm  = 1/ lag ** 2
    def komunitate_bakoitza(i):
        l = sum(w for k, t, w in G.edges(i, data=weight, default = 1) if t in i) #i komunitateko kide guztiek komunitatekoekin duten konekzion pisuen batura
        out = sum(degrees[k] for k in i) #Komunitateko nodo guztiek duten pisuaren balioaren batura
        return l / m - out**2 * norm
    
    return sum(map(komunitate_bakoitza, partizioa))

In [482]:
def lehenengoakLortu(num_part):
    n = random.randint(num_part, num_part * 5)
    aukerak = list(range(1, num_part +1))
    l = np.array(sorted(G.degree, key = lambda x: x[1], reverse = True))
    l = l[0: n]
    a = np.array(list(map(int, l[:, 1])))
    
    suma = sum(a)
    prob = a / suma
    return l[np.random.choice(np.arange(0, n), p=prob, size =num_part, replace = False)]

def eraikitzailea(G, num_part):
    authors = np.array(G.nodes())
    
    lehenengoak = lehenengoakLortu(num_part)
    partizioak =[[] for i in range(num_part +1)]
    #print(len(authors))
    des = []
    for i in range(len(lehenengoak)):
        idx =lehenengoak[i][0]
        partizioak[i].append(idx)
        des.append(nx.descendants(G, idx))
    
    for i in authors:
        aurkitu = False
        idx = 0
        for idj, j in enumerate(des):
            if i in j:
                idx = idj
                aurkitu = True
        if not aurkitu :
            idx = -1
        if i not in lehenengoak:
        
            partizioak[idx].append(i)
    gehituta = True
    
    return partizioak

#ERAIKITZAILEA
l = eraikitzailea(G, 5)
modularitatea(G, l)

0.5687811560048526

In [491]:
import random
import numpy as np
def randomSol(min_part = 2, max_part = 4):
    num_part = random.randint(min_part, max_part)
    sol = [[] for i in range(num_part)]
    
    for i in G:
        sol[random.randint(0, num_part -1)].append(i)
    sol = [set(sol[i]) for i in range(num_part)]
    return sol

def random_search(G, sol_kop, num_part):
    size = len(G)
    used_list = []
    best_fitness = -1000
    best_solution = []
    
    solutions = []
    for i in range(sol_kop):
        solutions.append(randomSol(min_part =num_part, max_part = num_part))
    fitness = list(map(lambda x: modularitatea(G, x), solutions))
    fitness.sort(reverse=True)
    best_fitness = max(fitness)
    best_solution = solutions[fitness.index(best_fitness)]
    return (best_fitness, best_solution)

fitness, l = random_search(G, 1000, 2)
modularitatea(G, l)

-0.0063514608200520295

In [472]:
"""
import numpy as np
import copy
def local_search(G,sol, max_evals, neigh_func):
    A = kalkulatu_A(G)
    size = len(G)
    best_solution = sol
    best_fitness = modularitatea(G,best_solution,A)
    evals = 1
    hobetu = True
    while hobetu and evals<max_evals:
        nei = neigh_func(best_solution,G)
        last_f = modularitatea(G,best_solution,A)
        for i in nei:
            cur_fit = modularitatea(i)
            evals +=1
            if cur_fit>best_fitness:
                best_solution = i
                best_fitness = cur_fit
                break
        if last_f == best_fitness:
            hobetu = False
            
    return (best_fitness, best_solution,evals)
def com_person(soluzioa, person):
    print(person)

    for idx, i in enumerate(soluzioa):
        if person in i:
            return idx
    print("gaixki")
    return 0

def ingurunea2(solution, G, size = 200):
    num_parts = len(solution)
    res = []
    for i in range(size):
        sol = solution.copy()
        print(solution[i])
        n = np.random.choice(range(num_parts), 2, replace=False)
        m = [np.random.choice(range(len(solution[i])), 2, replace=False) for i in n]
        sol[n[0]][m[0]] = sol[n[1]][m[1]]
        sol[n[1]][m[1]] = solution[n[0]][m[0]]
        res.append(sol)
    return res
a = eraikitzailea(G, 2)
print(ingurunea2(a, G))

def VND(G,initial_sol, max_evals):
    # BETE HEMEN 15-20 lerro
    A = kalkulatu_A(G)
    best_solution = initial_sol
    best_fitness = modularitatea(G,best_solution,A)
    cur_evals=1
    hobetu = True
    evals2 = float('inf')
    while cur_evals<max_evals:
        best_fitness,best_solution,evals_1 = local_search(G, best_solution, max_evals,A, ingurunea1)
        if evals_1 < evals_2:
            break
        cur_evals +=evals1
        erdiko_fit = best_fitness
        best_fitness,best_solution,evals_2 = local_search(G,best_solution, max_evals,A, ingurune2)
        cur_evals +=evals2
        if evals_2<=evals_1:
            break
    return (best_fitness, best_solution, cur_evals)


"""

'\nimport numpy as np\nimport copy\ndef local_search(G,sol, max_evals, neigh_func):\n    A = kalkulatu_A(G)\n    size = len(G)\n    best_solution = sol\n    best_fitness = modularitatea(G,best_solution,A)\n    evals = 1\n    hobetu = True\n    while hobetu and evals<max_evals:\n        nei = neigh_func(best_solution,G)\n        last_f = modularitatea(G,best_solution,A)\n        for i in nei:\n            cur_fit = modularitatea(i)\n            evals +=1\n            if cur_fit>best_fitness:\n                best_solution = i\n                best_fitness = cur_fit\n                break\n        if last_f == best_fitness:\n            hobetu = False\n            \n    return (best_fitness, best_solution,evals)\ndef com_person(soluzioa, person):\n    print(person)\n\n    for idx, i in enumerate(soluzioa):\n        if person in i:\n            return idx\n    print("gaixki")\n    return 0\n\ndef ingurunea2(solution, G, size = 200):\n    num_parts = len(solution)\n    res = []\n    for i 

In [488]:
def dict_to_part(sol, num_parts):
    part = [list() for i in range(num_parts + 1)]
    for key, val in sol.items():
        part[val].append(key)
    for i in range(len(part)):
        part[i] = set(part[i])
    return part
def part_to_dict(solution):
    parts = {}
    for i in range(len(solution)):
        for j in solution[i]:
            parts[j] = i
    return parts
def ingurunea2(solution, G):
    authors = np.array(G.nodes())

    num_parts = len(solution)
    res = []
    parts  = part_to_dict(solution)
    for i in authors:
        lag = parts
        for j in G.neighbors(i):
            lag[j] = lag[i]
        lag = dict_to_part(lag, num_parts)
        res.append(lag)
    
    return res


In [521]:
import sys
def simulated_annealing(G,part_kop, iter_kop, temp,beta):
    p  =0.75
    minFit, bbFit, maxFit = kalkulatuMaxMin(part_kop)
    t = (maxFit - minFit)/p
    best = randomSol(part_kop, part_kop)
    best_eval = modularitatea(G,best)
    curr, curr_eval = best, best_eval
    for i in range(iter_kop):
        # take a step
        candidates = ingurunea2(curr,G)
        candidate = candidates[np.random.randint(0,len(candidates))]
        candidate_eval = modularitatea(G,candidate)
        if candidate_eval > best_eval:
            best, best_eval = candidate, candidate_eval
        # uneko eta soluzio berriaren arteko diferentzia kalkulatu
        dif = curr_eval - candidate_eval
        # iterazio kopuruaren uneko temperatura lortu
        t = max(sys.float_info.min, t * beta)
        # metropolis acceptance criterion kalkulatu
        accept = exp(-dif*100 / t)
        r = rand()
        # check if we should keep the new point
        if dif < 0 or r < accept:
          # store the new current point
          curr, curr_eval = candidate, candidate_eval
    return [curr, curr_eval]
l = simulated_annealing(G, 2, 500, 5, 0.5)
def kalkulatuMaxMin(part_kop):
    n = 5000
    solutions = [randomSol(min_part =part_kop, max_part = part_kop) for i in range(n)]
    fitness = list(map(lambda x: modularitatea(G, x), solutions))
    return min(fitness), sum(fitness)/n, max(fitness)

0.09634076615208687


/tmp/ipykernel_13656/3826295928.py:22: RuntimeWarning: overflow encountered in exp
  accept = exp(-dif*100 / t)


In [ ]:
def EDA_Hasieraraketa(G,pop_tam,part_kop):
    pop = list()
    for i in range(pop_tam):
        pop.append(part_to_dict(randomSol(part_kop, part_kop))) #Partizio random bat sortu
    return pop


def EDA_ProbabilitateakLortu(populazioa, part_kop, authors):
    prob_list = [np.zeros(len(populazioa[0])) for i in range(part_kop)]
    for i in populazioa:
        
        for j in i:
            prob_list[i[j]][np.where(authors == j)] += 1 /len(populazioa)
    
    #print(populazioa)
    prob_list = np.array(prob_list)
    return prob_list 
def EDA_HurrengoBelaunaldiaSortu(prob_list, tamaina, part_kop):
    l = []
    res = []
    for i in range(len(prob_list[0])):
        l.append(np.random.choice(part_kop,tamaina, p = prob_list[:, i]))
    l = np.transpose(np.array(l))
    for i in range(len(l)):
        res.append(dict(zip(authors, l[i])))
    return res
def EDA_OnenakAukeratu(G, pop, tamaina, num_parts):
    pop_berri = sorted(pop, key=lambda x : modularitatea(G, dict_to_part(x, num_parts)), reverse=True)
    pop_berri = pop_berri[0:int(tamaina)]
    return pop_berri


def EDA(G,generazioak,populazio_tamaina,part_kop):
    authors = np.array(G.nodes())

    pop = EDA_Hasieraraketa(G,populazio_tamaina,part_kop)
    for i in range(generazioak):
        onenak = EDA_OnenakAukeratu(G, pop, populazio_tamaina/4, part_kop)
        
        print(i, "garren generazioko soluzio onena:", modularitatea(G, dict_to_part(onenak[0], part_kop)))
        prob_list = EDA_ProbabilitateakLortu(onenak, part_kop, authors)
        pop = EDA_HurrengoBelaunaldiaSortu(prob_list, populazio_tamaina, part_kop)
    
l = EDA(G, 100, 200, 2)

0 garren generazioko soluzio onena: 0.03005846516558841
1 garren generazioko soluzio onena: 0.0237870141265073
2 garren generazioko soluzio onena: 0.026492962898671785
3 garren generazioko soluzio onena: 0.02655899752893176
4 garren generazioko soluzio onena: 0.02537102799247201
5 garren generazioko soluzio onena: 0.03947448856669222
6 garren generazioko soluzio onena: 0.04885304058416459
7 garren generazioko soluzio onena: 0.045218643276023734
8 garren generazioko soluzio onena: 0.05401109708692475
9 garren generazioko soluzio onena: 0.06374662839273346
10 garren generazioko soluzio onena: 0.07115999635175008
11 garren generazioko soluzio onena: 0.07317286754728095
12 garren generazioko soluzio onena: 0.07785564633682693
13 garren generazioko soluzio onena: 0.09460208577898477
14 garren generazioko soluzio onena: 0.09739155855130482
15 garren generazioko soluzio onena: 0.10344124601463031
16 garren generazioko soluzio onena: 0.10579908277244754
17 garren generazioko soluzio onena: 0.1